<a href="https://colab.research.google.com/github/lweislo/CNResults/blob/master/HTML_letour_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This page lets you scrape results directly from any LeTour.fr-formatted race.
# Please follow these instructions!
* Shift-enter each cell or click play button by hovering over the [  ] above the start() cell to load the code
* After running the cells above start(), you can just run the start() cell alone multiple times.
* Enter the full URL including the stage number and http:// in at the prompt (see example)
* The file will directly download as aso_results.csv to your default downloads folder
* Open CSV file in TextMate or other text editor, change commas to tabs then paste into Excel for best results with accents.

* email laura.weislo@futurenet.com if something breaks!



In [11]:
import requests
import html5lib
import pandas as pd
from bs4 import BeautifulSoup
import os
import json
import numpy as np
import datetime
from google.colab import files

In [12]:
# Enter bib find/replace here
race_bibs = {1:"Egan Arley Bernal Gomez (Col) Ineos Grenadiers",
2:"Andrey Amador (CRc) Ineos Grenadiers",
3:"Richard Carapaz (Ecu) Ineos Grenadiers",
4:"Jonathan Castroviejo Nicolas (Spa) Ineos Grenadiers",
5:"Michal Kwiatkowski (Pol) Ineos Grenadiers",
6:"Luke Rowe (GBr) Ineos Grenadiers",
7:"Pavel Sivakov (Rus) Ineos Grenadiers",
8:"Dylan Van Baarle (Ned) Ineos Grenadiers",
11:"Primoz Roglic (Slo) Team Jumbo-Visma",
12:"George Bennett (NZl) Team Jumbo-Visma",
13:"Amund Grøndahl Jansen (Nor) Team Jumbo-Visma",
14:"Tom Dumoulin (Ned) Team Jumbo-Visma",
15:"Robert Gesink (Ned) Team Jumbo-Visma",
16:"Sepp Kuss (USA) Team Jumbo-Visma",
17:"Tony Martin (Ger) Team Jumbo-Visma",
18:"Wout van Aert (Bel) Team Jumbo-Visma",
21:"Peter Sagan (Svk) Bora-Hansgrohe",
22:"Emanuel Buchmann (Ger) Bora-Hansgrohe",
23:"Felix Grossschartner (Aut) Bora-Hansgrohe",
24:"Lennard Kämna (Ger) Bora-Hansgrohe",
25:"Gregor Mühlberger (Aut) Bora-Hansgrohe",
26:"Daniel Oss (Ita) Bora-Hansgrohe",
27:"Lukas Pöstlberger (Aut) Bora-Hansgrohe",
28:"Maximilian Schachmann (Ger) Bora-Hansgrohe",
31:"Romain Bardet (Fra) AG2R la Mondiale",
32:"Mikael Cherel (Fra) AG2R la Mondiale",
33:"Benoit Cosnefroy (Fra) AG2R la Mondiale",
34:"Pierre Latour (Fra) AG2R la Mondiale",
35:"Oliver Naesen (Bel) AG2R la Mondiale",
36:"Nans Peters (Fra) AG2R la Mondiale",
37:"Clement Venturini (Fra) AG2R la Mondiale",
38:"Alexis Vuillermoz (Fra) AG2R la Mondiale",
41:"Julian Alaphilippe (Fra) Deceuninck-Quickstep",
42:"Kasper Asgreen (Den) Deceuninck-Quickstep",
43:"Sam Bennett (Irl) Deceuninck-Quickstep",
44:"Rémi Cavagna (Fra) Deceuninck-Quickstep",
45:"Tim Declercq (Bel) Deceuninck-Quickstep",
46:"Dries Devenyns (Bel) Deceuninck-Quickstep",
47:"Bob Jungels (Lux) Deceuninck-Quickstep",
48:"Michael Mørkøv (Den) Deceuninck-Quickstep",
51:"Thibaut Pinot (Fra) Groupama-FDJ",
52:"William Bonnet (Fra) Groupama-FDJ",
53:"David Gaudu (Fra) Groupama-FDJ",
54:"Stefan Küng (Swi) Groupama-FDJ",
55:"Matthieu Ladagnous (Fra) Groupama-FDJ",
56:"Valentin Madouas (Fra) Groupama-FDJ",
57:"Rudy Molard (Fra) Groupama-FDJ",
58:"Sébastien Reichenbach (Swi) Groupama-FDJ",
61:"Mikel Landa Meana (Spa) Bahrain McLaren",
62:"Pello Bilbao (Spa) Bahrain McLaren",
63:"Damiano Caruso (Ita) Bahrain McLaren",
64:"Sonny Colbrelli (Ita) Bahrain McLaren",
65:"Marco Haller (Aut) Bahrain McLaren",
66:"Matej Mohoric (Slo) Bahrain McLaren",
67:"Wout Poels (Ned) Bahrain McLaren",
68:"Rafael Valls Ferri (Spa) Bahrain McLaren",
71:"Rigoberto Uran (Col) EF Pro Cycling",
72:"Alberto Bettiol (Ita) EF Pro Cycling",
73:"Hugh Carthy (GBr) EF Pro Cycling",
74:"Sergio Andres Higuita Garcia (Col) EF Pro Cycling",
75:"Jens Keukeleire (Bel) EF Pro Cycling",
76:"Daniel Felipe Martinez Poveda (Col) EF Pro Cycling",
77:"Neilson Powless (USA) EF Pro Cycling",
78:"Tejay Van Garderen (USA) EF Pro Cycling",
81:"Nairo Quintana (Col) Team Arkea-Samsic",
82:"Winner Anacona (Col) Team Arkea-Samsic",
83:"Warren Barguil (Fra) Team Arkea-Samsic",
84:"Kevin Ledanois (Fra) Team Arkea-Samsic",
85:"Dayer Quintana (Col) Team Arkea-Samsic",
86:"Diego Rosa (Ita) Team Arkea-Samsic",
87:"Clément Russo (Fra) Team Arkea-Samsic",
88:"Connor Swift (GBr) Team Arkea-Samsic",
91:"Alejandro Valverde (Spa) Movistar Team",
92:"Dario Cataldo (Ita) Movistar Team",
93:"Imanol Erviti (Spa) Movistar Team",
94:"Enric Mas Nicolau (Spa) Movistar Team",
95:"Nelson Oliveira (Por) Movistar Team",
96:"José Rojas (Spa) Movistar Team",
97:"Marc Soler (Spa) Movistar Team",
98:"Carlos Verona Quintanilla (Spa) Movistar Team",
101:"Richie Porte (Aus) Trek-Segafredo",
102:"Niklas Eg (Den) Trek-Segafredo",
103:"Kenny Elissonde (Fra) Trek-Segafredo",
104:"Bauke Mollema (Ned) Trek-Segafredo",
105:"Mads Pedersen (Den) Trek-Segafredo",
106:"Toms Skujins (Lat) Trek-Segafredo",
107:"Jasper Stuyven (Bel) Trek-Segafredo",
108:"Edward Theuns (Bel) Trek-Segafredo",
111:"Greg Van Avermaet (Bel) CCC Team",
112:"Alessandro De Marchi (Ita) CCC Team",
113:"Simon Geschke (Ger) CCC Team",
114:"Jan Hirt (Cze) CCC Team",
115:"Jonas Koch (Ger) CCC Team",
116:"Michael Schär (Swi) CCC Team",
117:"Matteo Trentin (Ita) CCC Team",
118:"Ilnur Zakarin (Rus) CCC Team",
121:"Guillaume Martin (Fra) Cofidis",
122:"Simone Consonni (Ita) Cofidis",
123:"Nicolas Edet (Fra) Cofidis",
124:"Jesus Herrada (Spa) Cofidis",
125:"Christophe Laporte (Fra) Cofidis",
126:"Anthony Perez (Fra) Cofidis",
127:"Pierre Luc Perichon (Fra) Cofidis",
128:"Elia Viviani (Ita) Cofidis",
131:"Tadej Pogacar (Slo) UAE Team Emirates",
132:"Fabio Aru (Ita) UAE Team Emirates",
133:"David De la Cruz Melgarejo (Spa) UAE Team Emirates",
134:"Davide Formolo (Ita) UAE Team Emirates",
135:"Alexander Kristoff (Nor) UAE Team Emirates",
136:"Vegard Stake Laengen (Nor) UAE Team Emirates",
137:"Marco Marcato (Ita) UAE Team Emirates",
138:"Jan Polanc (Slo) UAE Team Emirates",
141:"Miguel Angel Lopez Moreno (Col) Astana Pro Team",
142:"Omar Fraile Matarranz (Spa) Astana Pro Team",
143:"Hugo Houle (Can) Astana Pro Team",
144:"Gorka Izagirre Insausti (Spa) Astana Pro Team",
145:"Ion Izaguirre Insausti (Spa) Astana Pro Team",
146:"Alexey Lutsenko (Kaz) Astana Pro Team",
147:"Luis León Sanchez (Spa) Astana Pro Team",
148:"Harold Alfonso Tejada Canacue (Col) Astana Pro Team",
151:"Caleb Ewan (Aus) Lotto Soudal",
152:"Steff Cras (Bel) Lotto Soudal",
153:"Jasper De Buyst (Bel) Lotto Soudal",
154:"Thomas De Gendt (Bel) Lotto Soudal",
155:"John Degenkolb (Ger) Lotto Soudal",
156:"Frederik Frison (Bel) Lotto Soudal",
157:"Philippe Gilbert (Bel) Lotto Soudal",
158:"Roger Kluge (Ger) Lotto Soudal",
161:"Adam Yates (GBr) Mitchelton-Scott",
162:"Jack Bauer (NZl) Mitchelton-Scott",
163:"Samuel Bewley (NZl) Mitchelton-Scott",
164:"Esteban Chaves (Col) Mitchelton-Scott",
165:"Daryl Impey (RSA) Mitchelton-Scott",
166:"Christopher Juul Jensen (Den) Mitchelton-Scott",
167:"Luka Mezgec (Slo) Mitchelton-Scott",
168:"Mikel Nieve Iturralde (Spa) Mitchelton-Scott",
171:"Daniel Martin (Irl) Israel Start-Up Nation",
172:"André Greipel (Ger) Israel Start-Up Nation",
173:"Ben Hermans (Bel) Israel Start-Up Nation",
174:"Hugo Hofstetter (Fra) Israel Start-Up Nation",
175:"Krists Neilands (Lat) Israel Start-Up Nation",
176:"Guy Niv (Isr) Israel Start-Up Nation",
177:"Nils Politt (Ger) Israel Start-Up Nation",
178:"Tom Van Asbroeck (Bel) Israel Start-Up Nation",
181:"Niccolò Bonifazio (Ita) Total Direct Energie",
182:"Mathieu Burgaudeau (Fra) Total Direct Energie",
183:"Lilian Calmejane (Fra) Total Direct Energie",
184:"Jerome Cousin (Fra) Total Direct Energie",
185:"Fabien Grellier (Fra) Total Direct Energie",
186:"Romain Sicard (Fra) Total Direct Energie",
187:"Geoffrey Soupe (Fra) Total Direct Energie",
188:"Anthony Turgis (Fra) Total Direct Energie",
191:"Giacomo Nizzolo (Ita) NTT Pro Cycling",
192:"Edvald Boasson Hagen (Nor) NTT Pro Cycling",
193:"Ryan Gibbons (RSA) NTT Pro Cycling",
194:"Michael Gogl (Aut) NTT Pro Cycling",
195:"Michael Valgren (Den) NTT Pro Cycling",
196:"Roman Kreuziger (Cze) NTT Pro Cycling",
197:"Domenico Pozzovivo (Ita) NTT Pro Cycling",
198:"Maximilian Walscheid (Ger) NTT Pro Cycling",
201:"Tiesj Benoot (Bel) Team Sunweb",
202:"Nikias Arndt (Ger) Team Sunweb",
203:"Cees Bol (Ned) Team Sunweb",
204:"Marc Hirschi (Swi) Team Sunweb",
205:"Søren Kragh Andersen (Den) Team Sunweb",
206:"Joris Nieuwenhuis (Ned) Team Sunweb",
207:"Casper Phillip Pedersen (Den) Team Sunweb",
208:"Nicolas Roche (Irl) Team Sunweb",
211:"Bryan Coquard (Fra) B&B Hotels-Vital Concept",
212:"Cyril Barthe (Fra) B&B Hotels-Vital Concept",
213:"Maxime Chevalier (Fra) B&B Hotels-Vital Concept",
214:"Jens Debusschere (Bel) B&B Hotels-Vital Concept",
215:"Cyril Gautier (Fra) B&B Hotels-Vital Concept",
216:"Quentin Pacher (Fra) B&B Hotels-Vital Concept",
217:"Kevin Reza (Fra) B&B Hotels-Vital Concept",
218:"Pierre Rolland (Fra) B&B Hotels-Vital Concept",
"AG2R LA MONDIALE":"AG2R la Mondiale",
"ASTANA PRO TEAM":"Astana Pro Team",
"B&B HOTELS - VITAL CONCEPT P / B KTM":"B&B Hotels-Vital Concept",
"BAHRAIN - MCLAREN":"Bahrain McLaren",
"BORA - HANSGROHE":"Bora-Hansgrohe",
"CCC TEAM":"CCC Team",
"COFIDIS":"Cofidis",
"DECEUNINCK - QUICK - STEP":"Deceuninck-Quick-Step",
"EF PRO CYCLING":"EF Pro Cycling",
"GROUPAMA - FDJ":"Groupama-FDJ",
"ISRAEL START-UP NATION":"Israel Start-Up Nation",
"LOTTO SOUDAL":"Lotto Soudal",
"MITCHELTON - SCOTT":"Mitchelton-Scott",
"MOVISTAR TEAM":"Movistar Team",
"NTT PRO CYCLING TEAM":"NTT Pro Cycling",
"TEAM ARKEA - SAMSIC":"Team Arkea-Samsic",
"TEAM JUMBO - VISMA":"Team Jumbo-Visma",
"TEAM SUNWEB":"Team Sunweb",
"TOTAL DIRECT ENERGIE":"Total Direct Energie",
"TREK - SEGAFREDO":"Trek-Segafredo",
"UAE TEAM EMIRATES":"UAE Team Emirates",
"INEOS GRENADIERS":"Ineos Grenadiers"}

# ASO code for Tour classifications in page source
tab_dict = {'ite':'Stage',
'ipe':'Points',
'ime':'Mountains',
'ije':'Young riders',
'ice':'Combativity',
'ete':'Teams',
'itg':'General Classification',
'ipg':'Points Classification',
'img':'Mountains Classification',
'ijg':'Young Riders Classification',
'icg':'Combativity Classification',
'etg':'Teams Classification'}

order_list = ['ite', 'ipe', 'ime', 'ije', 'ice', 'ete', 'itg', 'ipg', 'img', 'icg','ijg', 'etg']
points_stg = ['ipe', 'ime']

In [13]:
# ALL THE CODE YOU NEED

def output_file(header_list, table_list):
    outfile = 'html_results.txt'
    with open(outfile, 'w') as file:
        for item in range(0, len(header_list)):
            # convert the tables to HTML and insert the headers as captions before the table head
            tmp_df = table_list[item].to_html(index=False)
            output_html = tmp_df.replace('<thead>',f'<caption>{header_list[item]}</caption><thead>')
            try:
                file.write(output_html)
            except:
                pass
    files.download(outfile)
# Scrape the page for stage and extracts a list of raw URLs coded with the classification code

def scrape_urls(stage_url):
    base_url = stage_url.split('/')[2]
    links_list = [] #Temporarily store the list of unparsed URLs
    url_dict = {} #Dictionary with result type code and full URL
    print(f"Getting result links for {stage_url}")

    page = requests.get(stage_url)

    if page.status_code == 200:
        content = page.content
        soup = BeautifulSoup(content, "html5lib")

    #Pull out a specific block of code with two sets of coded URLs from the soup. YMMV if ASO change the format!
        try:
            all_links = soup.find(class_="js-tabs-nav") #This is from a UL tag above the table on the page
            links = all_links.find_all(class_="js-tabs-ranking") #There are two items in the list, stage and GC
            
            for item in links:
                urls = item.get('data-ajax-stack')
                links_list.append(urls)

        except ElementDoesNotExist as e:
            print(f"That does not appear to be a valid results URL. {e}")
# Parse out the URLs from the list 'data-ajax-stack' attribute


    for item in links_list:
          #clean up the code into a useable URL
        urls = item.replace('\'', '"')
        myurl = urls.replace('\/', '/')
        myurls = json.loads(myurl)
        for key, value in myurls.items():
            url_dict[key] = f"https://{base_url}{value}"
            
    for item in order_list:   # IMPORTANT: We want to process these in the order of the universal result order list 
        try:
            # print(f"Scraping results from {tab_dict[item], url_dict[item]}")
            scrape_results(item, url_dict[item], stage_url)
        except KeyError:
            print(f"No results for {tab_dict[item]}")
#             output_tables.append("")
            pass

def scrape_results(item, url, stage_url):
    print(f"Getting results for {item}, {url}")
    out_df = pd.DataFrame()

    try:
        page = requests.get(url).content
        soup = BeautifulSoup(page, "html5lib")

#Treat sprint and mountains stage results differently, they have multiple tables per page and need to iterate through
        if item == 'ipe' or item == 'ime':
            if item == 'ipe':
                name = "Sprint"
            else:
                name = "Mountain"
            print(f"Found a {name}")
            mydiv = soup.find(class_="tabs__content")
            labels = mydiv.find_all(class_="rankingTables__caption")
            tables = mydiv.find_all(class_="rankingTable")

            for item in labels:
                try:
                    this_label = item.text.title()
                    this_label = this_label.replace(" Km", "").replace(" - "," km. ")
                    this_label = (f"{name} {labels.index(item)+1} - {this_label}")
                    print(this_label)
                    output_labels.append(this_label)
                except ValueError:
#                   output_labels.append("")
                    pass

            for item in tables:
                out_df = pd.DataFrame()
                try:
                    table = pd.read_html(str(item), index_col=None, header=None)
                    table = table[0]
                    out_df['Place'] = table['Rank']
                    out_df['Rider (Country) Team'] = table['Rider No.'].map(race_bibs).fillna(table['Rider No.'])
                    out_df['Result'] = table['Points'].str.replace('PTS', '')

                    output_tables.append(out_df)
                except ValueError:
                    print("Value error scrape")
                    pass        
        else:
            # Fish the tables out of the soup
            res_table = pd.read_html(page)

            df = res_table[0]
            # Get the heading for output
            label = tab_dict[item]
            output_labels.append(label)
            # Get the table for output, formatting points and times differently
            # TIMED CLASSIFICATIONS
            if item in ['ite', 'ije', 'itg', 'ijg']:
                # Convert the formatting for times
                df['Gap'] = df['Gap'].str.replace('-','').str.replace(' ','').str.replace('+','').str.replace('h', ':').str.replace('\'\'', '').str.replace('\'',':')
                df['Times'] = df['Times'].str.replace(' ','').str.replace('h', ':').str.replace('\'\'', '').str.replace('\'',':')
                # Grab the winning time
                wtime = df['Times'].iloc[0]
                df['Gap'].iloc[0] = wtime #make the number one the winner's time
                # Create the output dataframe
                out_df['Place'] = df['Rank'].fillna('')
                out_df['Rider (Country) Team'] = df['Rider No.'].map(race_bibs).fillna(df['Rider No.'])
                out_df['Result'] = df['Gap']

#                 For the stage results, go to the withdrawals function and append any for this stage to the end
                if item == 'ite':
                    try:
                        df = withdrawals(stage_url)
                        print(df)
                        out_df = out_df.append(df)
                    except:
                        pass
            elif item in ['ipg', 'img']:
                out_df['Place'] = df['Rank'].fillna('')
                out_df['Rider (Country) Team'] = df['Rider No.'].map(race_bibs).fillna(df['Rider No.'])
                out_df['Result'] = df['Points'].str.replace(' PTS', '')

            elif item == 'ice':
                out_df['Place'] = df['Rank'].fillna('')
                out_df['Rider (Country) Team'] = df['Rider No.'].map(race_bibs).fillna(df['Rider No.'])
                out_df['Result'] = ''

            elif item in ['ete', 'etg']:
                try:
                   # Convert the formatting for times
                    df['Gap'] = df['Gap'].str.replace('-','').str.replace(' ','').str.replace('+','').str.replace('h', ':').str.replace('\'\'', '').str.replace('\'',':')
                    df['Times'] = df['Times'].str.replace(' ','').str.replace('h', ':').str.replace('\'\'', '').str.replace('\'',':')
                    # Grab the winning time
                    wtime = df['Times'].iloc[0]
                    df['Gap'].iloc[0] = wtime #make the number one the winner's time
                    # Create the output dataframe
                    out_df['Place'] = df['Rank'].fillna('')
                    out_df['Rider (Country) Team'] = df['Team'].map(race_bibs).fillna(df['Team'])
                    out_df['Result'] = df['Gap']
                except TypeError:
                   # I don't even know why
                    pass
                    # df['Gap'] = df['Gap'].str.replace('-','').str.replace(' ','').str.replace('+','').str.replace('h', ':').str.replace('\'\'', '').str.replace('\'',':')
                    # df['Times'] = df['Times'].str.replace(' ','').str.replace('h', ':').str.replace('\'\'', '').str.replace('\'',':')
                    # # Grab the winning time
                    # wtime = df['Times'].iloc[0]
                    # df['Gap'].iloc[0] = wtime #make the number one the winner's time
                    # # Create the output dataframe
                    # out_df['Place'] = df['Rank'].fillna('')
                    # out_df['Rider (Country) Team'] = df['Team'].map(race_bibs).fillna(df['Rider No.'])
                    # out_df['Result'] = df['Gap']
            output_tables.append(out_df)


    except ValueError:
        print("No table found for " + item)  
        
        
def withdrawals(stage_url):
    print("Getting WD")
    base_url = stage_url.split('/')[2]
    stage_id = stage_url.split('/')[-1].title().replace('-', ' ') # Results in Stage X, the text inside div tag for that stage on withdrawals page
    wd_url = f"https://{base_url}/en/withdrawal" #YMMV IF ASO CHANGE THEIR SITE STRUCTURE
    print(f'Getting withdrawals for stage {stage_id}')

    page = requests.get(wd_url)
    out_df = pd.DataFrame() # Pick out the columns we want and put in a new dataframe to append    
    if page.status_code == 200:
        content = page.content
        soup = BeautifulSoup(content, "html5lib")
        divs = soup.find_all('div', class_="rankingTables__item") # Parent of each stage's withdrawal list
        wds = soup.find('div', text=stage_id).parent # Pull out only the table for the stage we want
        wdt = wds.find('table', class_="rankingTable rtable") # Pull the table out of the div
        wdt = pd.read_html(str(wdt))[0] # Format the table as a dataframe
        out_df['Place'] = wdt['Withdraw'].str.replace('withdrawal','DNF').str.replace('eliminated','DSQ').str.replace('outside the time limit','HD')
        out_df['Rider (Country) Team'] = wdt['Bib'].map(race_bibs).fillna(wdt['Bib'])
        out_df['Result'] = ''
    print(out_df)
    
    stage_id = ""
    return out_df # Returns this to the function line 95

def start():

    stage_url = input("Enter the URL to LeTour results including stage id rankings/stage-5 : ")
    scrape_urls(stage_url)
    output_file(output_labels, output_tables)

## THIS STARTS THE PROGRAM RUNNING
* You only need to run all the cells above once before clicking the cell below to start extracting results

In [14]:
output_labels = []
output_tables = []
start()
# https://www.criterium-du-dauphine.fr/en/rankings/stage-1

KeyboardInterrupt: ignored